In [ ]:
#Import
import requests
import lxml.html as lh
import pandas as pd

In [3]:
#Get info from wikipedia
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store data and parse data between tr of HTML
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [4]:
#Check length
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [5]:
# Parse the first row as our header
tr_elements = doc.xpath('//tr')


col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighborhood
"


In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
#Create dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [8]:
#Drop cells with 'Not assigned'
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods\n
1,M4A,North York,Victoria Village\n
2,M5A,Downtown Toronto,Harbourfront\n
3,M6A,North York,Lawrence Heights\n
4,M6A,North York,Lawrence Manor\n
5,M7A,Downtown Toronto,Queen's Park\n
6,M9A,Queen's Park,Not assigned\n
7,M1B,Scarborough,Rouge\n
8,M1B,Scarborough,Malvern\n
9,M3B,North York,Don Mills North\n


In [9]:
#Groupby Postcode and Borough
df = df.groupby(['Postcode', 'Borough'])['Neighborhood\n'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighborhood\n']
df.head(10)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge\n,Malvern\n"
1,M1C,Scarborough,"Highland Creek\n,Rouge Hill\n,Port Union\n"
2,M1E,Scarborough,"Guildwood\n,Morningside\n,West Hill\n"
3,M1G,Scarborough,Woburn\n
4,M1H,Scarborough,Cedarbrae\n
5,M1J,Scarborough,Scarborough Village\n
6,M1K,Scarborough,"East Birchmount Park\n,Ionview\n,Kennedy Park\n"
7,M1L,Scarborough,"Clairlea\n,Golden Mile\n,Oakridge\n"
8,M1M,Scarborough,"Cliffcrest\n,Cliffside\n,Scarborough Village W..."
9,M1N,Scarborough,"Birch Cliff\n,Cliffside West\n"


In [10]:
#Assighn Borough value to Neighbourhood "Not Assigned"
df.loc[df['Neighborhood\n'] == 'Not assigned', 'Neighborhood\n'] = df['Borough']

In [11]:
#Double Check
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighborhood
93,M9A,Queen's Park,Not assigned\n


In [12]:
#Check Shape
df.shape

(103, 3)

In [13]:
df.to_csv(r'df_can.csv')